In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
# from mne.decoding import SPoC
from tensorflow.keras.layers import GlobalAveragePooling1D, Attention, Dense, Flatten, Conv2D, MaxPooling2D, Conv1D,MaxPool2D,GRU, Dropout, MaxPool1D, ConvLSTM2D,MaxPooling1D, ReLU, LeakyReLU, RNN, SimpleRNN, GRU, LSTM, TimeDistributed
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import KFold
from scipy.interpolate import interp1d
import tensorflow as tf
from scipy.signal import stft
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import h5py # Read and write HDF5 files from Python
import os
import matplotlib.pyplot as plt2
import matplotlib
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


data_path = "/kaggle/input/dreem-2-sleep-classification-challenge-2020/"
file_xtrain = data_path + "X_train.h5/X_train.h5"
file_xtest = data_path + "X_test.h5/X_test.h5"
file_ytrain = data_path + "y_train.csv"

In [ ]:


# training labels
# # what does the h5 file contains ?
# with h5py.File(file_xtrain, "r") as hf:
#        print(list(hf.keys()))

# # How to load data from h5? what is its shape and type?
# with h5py.File(file_xtrain, "r") as hf:
#        field = list(hf.keys())[0]
#        x_data = hf[field][()]
# type(x_data), x_data.shape

def normalize_data(eeg_array):
    """normalize signal between 0 and 1"""

    normalized_array = np.clip(eeg_array, -150, 150)
    normalized_array = normalized_array / 150

    return normalized_array

def split_data(input_signals_list,validation_ratio=0.3):
    with h5py.File(file_xtrain, "r") as fi:
        if len(input_signals_list) == 1:
            x_data = fi[input_signals_list[0]][()]
        else:
            x_data = np.zeros([24688,1500,len(input_signals_list)], dtype=np.float64)
            for i in range(0, len(input_signals_list)):
                if 'x' in input_signals_list[i] or 'y' in input_signals_list[i] or 'z' in input_signals_list[i]:
                    f1 = interp1d(np.arange(0, 300), fi[input_signals_list[i]][()], axis=1)
                    xnew = np.linspace(0, 30, num=1500)
                    x_data[0:24688, 0:1500, i] = f1(xnew)
                else:
                    x_data[0:24688, 0:1500, i] = fi[input_signals_list[i]][()]
        
        y_data = pd.read_csv(file_ytrain)['sleep_stage'].to_numpy()
        

        
        #x_data_to_connect = x_data[1:,:,:]
#         x_data = np.concatenate((x_data[0:-2,:,:],x_data[1:-1,:,:],x_data[2:,:,:]),axis=1)
#         for a in range(0,x_data.shape[0]-1):
#             x_data_new=np.zeros([x_data.shape[0]-1,3000,x_data.shape[2]])
#             x_data_new[a,:,:] = np.append(x_data[a,:,:],x_data[a+1,:,:],axis=0)
#         x_data = x_data_new
#         x_data_new = 0
        x_data = np.append(x_data[0:-1,:,:],x_data[1:,:,:],axis=1)
#        x_metadata = x_metadata[1:]
#        print(x_metadata.shape)
#        y_data =np.delete(y_data,0)
        # Creating data indices for training and validation splits:
#        dataset_size = len(y_data)
#        indices = list(range(dataset_size))
#        split = int((1 - validation_ratio) * dataset_size)
#        np.random.shuffle(indices)

        
        
#        train_indices, val_indices = np.argwhere(x_metadata%5 != 0) ,np.argwhere(x_metadata%5 == 0)
#        train_indices, val_indices =  np.squeeze(train_indices), np.squeeze(val_indices)


#        x_train, x_validation = x_data[train_indices], x_data[val_indices]
#        y_train, y_validation = y_data[train_indices], y_data[val_indices]

        x_data =  normalize_data(x_data)
        print('data ready')

    return x_data ,y_data
input_signals_list = ['eeg_4','eeg_5']
# input_signals_list = ['eeg_1','eeg_2','eeg_3','eeg_4','eeg_5','y']
#input_signals_list = ['eeg_3','eeg_4','eeg_5']
#input_signals_list = ['eeg_1','eeg_2','eeg_3','eeg_4','eeg_5','eeg_6','eeg_7']
# input_signals_list = ['eeg_1','eeg_2','eeg_3','eeg_4','eeg_5','eeg_6','eeg_7','x','y','z']
# input_signals_list = ['eeg_4','eeg_1']
# input_signals_list = [ 'eeg_3', 'eeg_4', 'eeg_5']
inputs, targets = split_data(input_signals_list)


In [ ]:
def custom_Kfold(n=5): 
    
    with h5py.File(file_xtrain, "r") as fi:
        x_metadata = fi['index'][()]
        
    
    
    indice = []
    
    for i in range(5): 
        train_indices, val_indices = np.argwhere(x_metadata%5 != 0+i) ,np.argwhere(x_metadata%5 == 0+i)


        indice.append(  {"train":np.squeeze(train_indices), "validation": np.squeeze(val_indices)})
    
    return(indice)


                      




        
        
    

In [ ]:
### Model configuration
batch_size = 32
loss_function = sparse_categorical_crossentropy
no_classes = 5
no_epochs = 20
optimizer = Adam(learning_rate=0.001)
# optimizer = SGD(learning_rate=0.01)
verbosity = 1
num_folds = 3

# Determine shape of the data

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
# inputs = np.concatenate((x_train, x_validation), axis=0)
# targets = np.concatenate((y_train, y_validation), axis=0)

# Define the K-fold Cross Validator
#kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

# no_epochs = 80
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') > 0.76):
            if (logs.get('val_loss') < 0.6):
                print("Reached 99% accuracy so cancelling trining!")
                self.model.stop_training = True

callbacks = myCallback()

def custom_loss(ytrue, ypred):
    
    
    scce = tf.keras.losses.SparseCategoricalCrossentropy()
    
#     weight =tf.constant([[0.85347313, 0.93098802, 0.6152054,  0.79114608, 0.80918737]])
    #weight =tf.constant([[0.00001, 1.0000, 0.00001, 0.00001, 0.001]])
    weight = tf.constant([[0.94355903, 2.70683738, 0.96141876, 0.55233263, 0.62986756]])
 
#     y_hot = tf.one_hot(ytrue, 5)
    

    
    new_y = tf.expand_dims(ypred, axis=1)


    new_weight = tf.matmul(weight,new_y, transpose_b = True)
    
    score = scce(ytrue,ypred, sample_weight= new_weight)
    
    return score



In [ ]:
# Model configuration
batch_size = 32
loss_function = sparse_categorical_crossentropy
no_classes = 5
no_epochs = 20
optimizer = Adam(learning_rate=0.001)
verbosity = 1
num_folds = 3
n_outputs=5
n_features=len(input_signals_list)
fold_no = 3
no_epochs = 20
# for train, test in kfold.split(inputs, targets):
# n_steps, n_length = 2, 1500
# trainX = inputs[train]
# testX = inputs[test]
# x_train, y_train, x_validation, y_validation
# trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
# testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
# x_train = x_train.reshape((x_train.shape[0], n_steps, 1, n_length, n_features))
# x_validation = x_validation.reshape((x_validation.shape[0], n_steps, 1, n_length, n_features))
# Determine shape of the data

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
#inputs = np.concatenate((x_train, x_validation), axis=0)
#targets = np.concatenate((y_train, y_validation), axis=0)

# Define the K-fold Cross Validator
#kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1


In [ ]:
 _, _, Zxx_train  = stft(inputs ,fs = 50, axis= 1)

Zxx_train = np.log(Zxx_train)
Zxx_train = np.clip(Zxx_train,-20,20)

Zxx_train = np.swapaxes(Zxx_train,1,2)
# Merge inputs and targets
inputs = Zxx_train


# Define the K-fold Cross Validator

# K-fold Cross Validation model evaluation
fold_no = 1
# checkpointer = [ModelCheckpoint(filepath= "mixed_mod_simplesect_1draw_otherchannel.hdf5",verbose=0, save_best_only=True), EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience = 10)]

In [ ]:

def custom_Kfold(n=5): 
    
    with h5py.File(file_xtrain, "r") as fi:
        x_metadata = fi['index'][()]
    
    x_metadata = x_metadata[1:]

        
    
    
    indice = []
    
    for i in range(5): 
        train_indices, val_indices = np.argwhere(x_metadata%5 != 0+i) ,np.argwhere(x_metadata%5 == 0+i)


        indice.append(  {"train":np.squeeze(train_indices), "validation": np.squeeze(val_indices)})
    
    return(indice)

dic_kfold = custom_Kfold()
                      


In [ ]:
number_channels = 2

for dic in dic_kfold: 
    train , test = dic.values()
    
    # Define the model architecture
    model = Sequential()
    model.add(tf.keras.layers.Reshape((number_channels, 25, 129),input_shape=(number_channels, 129, 25)))
    # model.add(TimeDistributed(Dense(128,input_shape=(7, 129, 25))))
    model.add(TimeDistributed(Dense(64)))
    model.add(tf.keras.layers.Reshape((number_channels, 64, 25)))
    model.add(tf.keras.layers.Reshape((25, 64, number_channels)))
    model.add(TimeDistributed(Dense(1)))
    model.add(tf.keras.layers.Reshape((1, 64, 25)))
    model.add(Dropout(0.25))
    model.add(tf.keras.layers.Reshape((1,25, 64)))
    model.add(TimeDistributed(tf.keras.layers.Bidirectional(GRU(128, go_backwards=False))))
#     model.add(ConvLSTM2D(filters=64, kernel_size=(1,7),padding='Same', return_sequences=True,input_shape=(n_steps, 1, n_length, n_features)))
#     model.add(LeakyReLU(alpha=0.1))
#     model.add(BatchNormalization())
#     model.add(ConvLSTM2D(filters=128, kernel_size=(1,3),padding='Same', go_backwards=True))
#     model.add(LeakyReLU(alpha=0.1))
#     model.add(BatchNormalization())
#     model.add(MaxPool2D(4,4))
    model.add(Flatten())
#     model.add(TimeDistributed(Flatten()))
#     model.add(tf.keras.layers.Bidirectional(LSTM(128,return_sequences=True)))
#     model.add(LeakyReLU(alpha=0.1))
#     model.add(tf.keras.layers.Bidirectional(LSTM(128,return_sequences=True)))
#     model.add(LeakyReLU(alpha=0.1))
#     model.add(LSTM(64))
#     model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(100))
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dense(n_outputs, activation='softmax'))

    # Compile the model
    model.compile(loss=custom_loss,
                optimizer=optimizer,
                metrics=['accuracy'])

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size, validation_data=(inputs[test], targets[test]),
              epochs=200,callbacks = [ModelCheckpoint(filepath= "model_"+str(fold_no)+".hdf5",verbose=0, save_best_only=True),
                                                     EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience = 10)],verbose=verbosity)
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
#     if (scores[0] < 0.57):
#         model.save('model_num_'+str(fold_no)+'.h5')
#         print("Saved model to disk")
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

def plot_heatmap(y_true,y_pred): 
    
    mat = confusion_matrix(y_true,y_pred)
    mat = mat/np.sum(mat ,axis =0)*100
    mat = mat.astype(np.int)

    fig, ax = plt.subplots()

    im, cbar = heatmap(mat, ['Wake','NREM1','NREM2','NREM3','REM'],['Wake','NREM1','NREM2','NREM3','REM'], ax=ax,
                       cmap="YlGn", cbarlabel="Pourcentage de classification")
    texts = annotate_heatmap(im, valfmt="{x:} %")

    fig.tight_layout()
    plt.show()
    
y_estimate = model.predict_classes(Zxx_validation)
plot_heatmap(y_validation, y_estimate)

In [ ]:
import winsound
duration = 1000  # milliseconds
freq = 600  # Hz
winsound.Beep(freq, duration)

In [ ]:
# from keras.models import load_model
# model = load_model('./model.h5',compile=True)
# del x_train
# del x_validation
model.load_weights("model_1.hdf5")
def test_data(input_signals_list):
    with h5py.File(file_xtest, "r") as fi:
        index = fi['index_absolute'][()]
        if len(input_signals_list) == 1:
            x_data = fi[input_signals_list[0]][()]
        else:
            x_data = np.zeros([24980,1500,len(input_signals_list)], dtype=np.float64)
            for i in range(0, len(input_signals_list)):
                if 'x' in input_signals_list[i] or 'y' in input_signals_list[i] or 'z' in input_signals_list[i]:
                    f1 = interp1d(np.arange(0, 300), fi[input_signals_list[i]][()], axis=1)
                    xnew = np.linspace(0, 30, num=1500)
                    x_data[0:24980, 0:1500, i] = f1(xnew)
                else:
                    x_data[0:24980, 0:1500, i] = fi[input_signals_list[i]][()]
        x_data = np.append(x_data[0:-1,:,:],x_data[1:,:,:],axis=1)
        x_test = normalize_data(x_data)
        return x_test, index
test_data,index = test_data(input_signals_list)
_, _, Zxx_test  = stft(test_data ,fs = 50, axis= 1)

Zxx_test = np.log(Zxx_test)
Zxx_test = np.clip(Zxx_test,-20,20)
Zxx_test = np.swapaxes(Zxx_test,1,2)
y_test = model.predict_classes(Zxx_test)
# df = pd.DataFrame(data={'index': index, 'sleep_stage': y_test})
y_test=np.append(y_test[0],y_test)

In [ ]:
if (index.shape) == (y_test.shape):
    df = pd.DataFrame(data={'index': index, 'sleep_stage': y_test})
    df.to_csv('our_result_GRU.csv',index=False)
    print('you can save the results')
else:
    print('there is an error in the shape of y_test')

In [ ]:
# for train, test in kfold.split(inputs, targets):

#     # Define the model architecture
#     model = Sequential()
#     model.add(TimeDistributed(Dense(128,input_shape=(129, 7, 25))))
#     model.add(TimeDistributed(tf.keras.layers.Bidirectional(GRU(128, go_backwards=False))))
# #     model.add(ConvLSTM2D(filters=64, kernel_size=(1,7),padding='Same', return_sequences=True,input_shape=(n_steps, 1, n_length, n_features)))
# #     model.add(LeakyReLU(alpha=0.1))
# #     model.add(BatchNormalization())
# #     model.add(ConvLSTM2D(filters=128, kernel_size=(1,3),padding='Same', go_backwards=True))
# #     model.add(LeakyReLU(alpha=0.1))
# #     model.add(BatchNormalization())
# #     model.add(MaxPool2D(4,4))
#     model.add(Flatten())
# #     model.add(TimeDistributed(Flatten()))
# #     model.add(tf.keras.layers.Bidirectional(LSTM(128,return_sequences=True)))
# #     model.add(LeakyReLU(alpha=0.1))
# #     model.add(tf.keras.layers.Bidirectional(LSTM(128,return_sequences=True)))
# #     model.add(LeakyReLU(alpha=0.1))
# #     model.add(LSTM(64))
# #     model.add(LeakyReLU(alpha=0.1))
#     model.add(Dropout(0.2))
#     model.add(Dense(256))
#     model.add(LeakyReLU(alpha=0.1))
#     model.add(Dense(100))
#     model.add(LeakyReLU(alpha=0.1))
#     model.add(Dense(n_outputs, activation='softmax'))

#     # Compile the model
#     model.compile(loss=custom_loss,
#                 optimizer=optimizer,
#                 metrics=['accuracy'])

#     # Generate a print
#     print('------------------------------------------------------------------------')
#     print(f'Training for fold {fold_no} ...')

#     # Fit data to model
#     history = model.fit(inputs[train], targets[train],
#               batch_size=batch_size, validation_data=(inputs[test], targets[test]),
#               epochs=200,callbacks = [ModelCheckpoint(filepath= "model_"+str(fold_no)+".hdf5",verbose=0, save_best_only=True),
#                                                      EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience = 10)],verbose=verbosity)
    
#     # Generate generalization metrics
#     scores = model.evaluate(inputs[test], targets[test], verbose=0)
# #     if (scores[0] < 0.57):
# #         model.save('model_num_'+str(fold_no)+'.h5')
# #         print("Saved model to disk")
#     print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
#     acc_per_fold.append(scores[1] * 100)
#     loss_per_fold.append(scores[0])

#     # Increase fold number
#     fold_no = fold_no + 1

# # == Provide average scores ==
# print('------------------------------------------------------------------------')
# print('Score per fold')
# for i in range(0, len(acc_per_fold)):
#   print('------------------------------------------------------------------------')
#   print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
# print('------------------------------------------------------------------------')
# print('Average scores for all folds:')
# print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
# print(f'> Loss: {np.mean(loss_per_fold)}')
# print('------------------------------------------------------------------------')